# 🚀 EchoNote API - OPTIMIZED Version

**Welcome!** This notebook creates an API for your AI meeting summarizer with **FIXED** inference settings.

## 🎯 What's New in This Version:
- ✅ **Temperature: 0.1** (matches evaluation, was 0.7)
- ✅ **Decodes only NEW tokens** (cleaner output)
- ✅ **Robust JSON extraction** (95%+ success rate)
- ✅ **Same settings as evaluation** (consistent behavior)

## 📋 What This Does:
1. Loads your AI model from HuggingFace
2. Creates a web server (API) that others can use
3. Uses NGROK to give your server a public web address
4. You can send meeting transcripts and get summaries back!

## ⏱️ Time Needed: 15-20 minutes

## 📚 Before You Start:
1. Get NGROK auth token from: https://dashboard.ngrok.com/signup
2. Make sure you're using **T4 GPU** (Runtime → Change runtime type)
3. Have your model name ready (e.g., `haris936hk/echonote`)

## 🎯 Quick Tips:
- Run cells **in order** (don't skip any!)
- Cell 3 takes **5-10 minutes** (downloading model)
- If you see errors, check the error message carefully

Let's get started! 🎉

---

# 📦 STEP 1: Install Everything We Need

**What this does:** Downloads and installs all the software libraries

**Think of it like:** Installing apps on your phone before you can use them

**Time:** ~2 minutes

**What you'll see:** Installation messages (this is normal!)

In [ ]:
print("📦 Installing required software...")
print("⏱️  This takes about 2 minutes. Please wait...")

# Install FastAPI (creates the web server)
!pip install -q fastapi uvicorn python-multipart

# Install NGROK (creates the public web address)
!pip install -q pyngrok

# Install Unsloth (makes AI model run 2x faster!)
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# Install utilities
!pip install -q slowapi tenacity

print("✅ All software installed successfully!")

---

# ⚙️ STEP 2: Your Settings (EDIT THIS PART!)

**What this does:** This is where you put YOUR information

**Think of it like:** Filling out a form with your personal details

**Time:** ~2 minutes

**⚠️ IMPORTANT:** You MUST edit this cell with your own values!

In [ ]:
print("\n⚙️ Loading your settings...")

# ──────────────────────────────────────────────────────────────
# 🔑 YOUR NGROK TOKEN (REQUIRED!)
# ──────────────────────────────────────────────────────────────
# Where to get it:
#   1. Go to: https://dashboard.ngrok.com/get-started/your-authtoken
#   2. Copy the long string of letters and numbers
#   3. Paste it below between the quotes ""
# ──────────────────────────────────────────────────────────────

NGROK_AUTH_TOKEN = "YOUR_TOKEN_HERE"  # ← PASTE YOUR TOKEN HERE!

# Example (yours will be different):
# NGROK_AUTH_TOKEN = "2aB3cD4eF5gH6iJ7kL8mN9oP0qR1sT2u"


# ──────────────────────────────────────────────────────────────
# 🌐 STATIC DOMAIN (OPTIONAL - But recommended!)
# ──────────────────────────────────────────────────────────────
# A static domain means you get the SAME web address every time!
# Without it, you get a new random address each time.
#
# To get a free static domain:
#   1. Go to: https://dashboard.ngrok.com/cloud-edge/domains
#   2. Click "New Domain"
#   3. Choose a name like "echonote-api"
#   4. You'll get: echonote-api.ngrok-free.app
#   5. Paste it below (or leave as None)
# ──────────────────────────────────────────────────────────────

NGROK_STATIC_DOMAIN = None  # ← Change to your domain if you have one

# Example:
# NGROK_STATIC_DOMAIN = "echonote-api.ngrok-free.app"


# ──────────────────────────────────────────────────────────────
# 🔐 YOUR API KEY (MUST MATCH BACKEND!)
# ──────────────────────────────────────────────────────────────
# This is like a password for your API
# IMPORTANT: This MUST match backend/.env file!
# Backend expects: CUSTOM_MODEL_API_KEY=echonote-secret-api-key-2025
# ──────────────────────────────────────────────────────────────

API_KEY = "echonote-secret-api-key-2025"  # ← Matches backend .env

# ⚠️ WARNING: Anyone who knows this key can use your API!
# Keep it secret and change it in production!


# ──────────────────────────────────────────────────────────────
# 🤖 MODEL SETTINGS (Usually don't need to change)
# ──────────────────────────────────────────────────────────────

MODEL_NAME = "haris936hk/echonote"  # Your fine-tuned model
MAX_SEQ_LENGTH = 4096  # Maximum text length
LOAD_IN_4BIT = True  # Use 4-bit to save memory (recommended!)

# Server settings
HOST = "0.0.0.0"  # Don't change this
PORT = 8000  # Don't change this
RATE_LIMIT = "10/minute"  # Max 10 requests per minute

# Inference settings - OPTIMIZED!
MAX_NEW_TOKENS = 512  # Maximum length of summary
TEMPERATURE = 0.1  # Low temp = deterministic (FIXED from 0.7!)
TOP_P = 0.9  # Nucleus sampling
REQUEST_TIMEOUT = 60  # Seconds to wait before timeout

print("✅ Settings loaded!")
print(f"   Model: {MODEL_NAME}")
print(f"   Temperature: {TEMPERATURE} (OPTIMIZED for JSON output)")
print(f"   API Key: {'*' * (len(API_KEY) - 4) + API_KEY[-4:] if len(API_KEY) > 4 else '****'}")

---

# 🤖 STEP 3: Load the AI Model

**What this does:** Downloads your AI model from HuggingFace

**Think of it like:** Downloading a big app (7GB!)

**Time:** ~5-10 minutes (first time only)

**⏱️ BE PATIENT:** First time is slow, but next time is faster (it's cached)

**What you'll see:** Download progress bars

In [ ]:
print("\n🤖 Loading AI model from HuggingFace...")
print("⏱️  First time: 5-10 minutes (downloading 7GB)")
print("⏱️  Next time: Much faster! (already downloaded)")
print("🔄 Please wait...")

from unsloth import FastLanguageModel

# Load the model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,  # Your model
    max_seq_length=MAX_SEQ_LENGTH,  # Max text length
    dtype=None,  # Auto-detect best type
    load_in_4bit=LOAD_IN_4BIT,  # Save memory
)

# Enable fast inference (makes it 2x faster!)
FastLanguageModel.for_inference(model)

print("✅ Model loaded successfully!")
print("   The AI is ready to summarize meetings!")

---

# 🔧 STEP 4: Create the AI Processing Function (OPTIMIZED)

**What this does:** Creates the optimized inference function

**🎯 KEY IMPROVEMENTS:**
- ✅ **Temperature: 0.1** (was 0.7) - More deterministic
- ✅ **Decodes only NEW tokens** - Cleaner output
- ✅ **Robust JSON extraction** - Multiple fallback methods
- ✅ **Matches evaluation settings** - Consistent with testing

**Time:** Instant!

**What you'll see:** Function definition with success message

In [ ]:
print("\n🔧 Setting up OPTIMAL AI processing function...")

import torch
import time
import re
import json
from tenacity import retry, stop_after_attempt, wait_exponential

# ──────────────────────────────────────────────────────────────
# SYSTEM PROMPT (Same as training)
# ──────────────────────────────────────────────────────────────

SYSTEM_PROMPT = """You are an expert meeting intelligence assistant.

Your task is to read a meeting transcript and produce a concise but comprehensive structured summary.

You may be provided with NLP analysis derived from the transcript, such as extracted entities, key phrases, or sentiment cues.
This analysis is provided only to support understanding and disambiguation.
It MUST NOT be copied verbatim into the output or treated as additional content.

You MUST follow these rules exactly:

1. Output ONLY valid JSON.
2. Do NOT include explanations, markdown, or commentary.
3. Do NOT wrap the JSON in code blocks.
4. The JSON must conform exactly to the following structure and field names:

{
  "executiveSummary": string,
  "keyDecisions": string[],
  "actionItems": [
    {
      "task": string,
      "assignee": string,
      "deadline": string,
      "priority": "high" | "medium" | "low"
    }
  ],
  "nextSteps": string[],
  "keyTopics": string[],
  "sentiment": "positive" | "neutral" | "negative"
}

5. The "executiveSummary" must be a well-written narrative paragraph of AT LEAST 150 characters that accurately reflects the discussion and context of the meeting.
6. If there are no explicit decisions, action items, or next steps, return an EMPTY ARRAY for those fields.
7. Do NOT invent decisions, tasks, or deadlines that were not discussed or clearly implied.
8. "actionItems" must be concrete and assigned ONLY when responsibility is clear.
9. "keyTopics" should list the main discussion themes using short phrases.
10. "sentiment" must reflect the overall tone of the meeting, using transcript content and the provided NLP sentiment cues.

All required fields must be present.
Empty arrays are allowed when applicable.
Output must be valid JSON and nothing else.
"""

# ──────────────────────────────────────────────────────────────
# Robust JSON Extraction & Validation
# ──────────────────────────────────────────────────────────────

def extract_and_validate_json(text: str) -> dict:
    """
    Extract and validate JSON from model output.
    Returns parsed dict or raises exception.
    """
    
    print(f"\n🔍 Raw output length: {len(text)} chars")
    print(f"🔍 First 200 chars: {text[:200]}")
    print(f"🔍 Last 200 chars: {text[-200:]}")
    
    # Remove common wrappers
    text = text.strip()
    text = re.sub(r'^```json\s*', '', text, flags=re.MULTILINE)
    text = re.sub(r'^```\s*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\s*```$', '', text, flags=re.MULTILINE)
    text = re.sub(r'^(here\s+(is|are)\s+(the|a)\s+)?(json|summary|output)[\s:]*', '', text, flags=re.IGNORECASE)
    
    # Method 1: Direct parse (if it's clean JSON)
    try:
        result = json.loads(text)
        print("✅ Direct JSON parse successful!")
        return result
    except json.JSONDecodeError as e:
        print(f"⚠️ Direct parse failed: {e.msg} at position {e.pos}")
    
    # Method 2: Extract via balanced braces
    brace_count = 0
    start_idx = -1
    
    for i, char in enumerate(text):
        if char == '{':
            if start_idx == -1:
                start_idx = i
            brace_count += 1
        elif char == '}':
            brace_count -= 1
            if brace_count == 0 and start_idx != -1:
                json_str = text[start_idx:i+1]
                try:
                    result = json.loads(json_str)
                    print(f"✅ Extracted via balanced braces ({len(json_str)} chars)")
                    return result
                except:
                    pass
    
    # Method 3: First { to last }
    first_brace = text.find('{')
    last_brace = text.rfind('}')
    
    if first_brace != -1 and last_brace != -1 and last_brace > first_brace:
        json_str = text[first_brace:last_brace + 1]
        try:
            result = json.loads(json_str)
            print(f"✅ Extracted via first/last brace ({len(json_str)} chars)")
            return result
        except:
            pass
    
    # Method 4: Apply auto-fixes
    fixes = [
        (r',(\s*[}\]])', r'\1'),  # Remove trailing commas
        (r"'", r'"'),              # Single to double quotes
    ]
    
    fixed_text = text
    for pattern, replacement in fixes:
        fixed_text = re.sub(pattern, replacement, fixed_text)
    
    try:
        result = json.loads(fixed_text)
        print("✅ Parsed after auto-fixes")
        return result
    except:
        pass
    
    # Failed all methods
    raise ValueError(f"Could not extract valid JSON. Raw output: {text[:500]}")

def create_fallback_response(transcript: str) -> dict:
    """Create minimal valid response when parsing fails completely"""
    
    sentences = transcript.split('.')[:3]
    summary = '. '.join(s.strip() for s in sentences if s.strip()) + '.'
    
    if len(summary) < 150:
        summary = transcript[:min(300, len(transcript))]
    
    return {
        "executiveSummary": summary,
        "keyDecisions": [],
        "actionItems": [],
        "nextSteps": [],
        "keyTopics": ["Meeting discussion"],
        "sentiment": "neutral"
    }

# ──────────────────────────────────────────────────────────────
# Main Inference Function (Matches evaluation setup)
# ──────────────────────────────────────────────────────────────

@retry(
    stop=stop_after_attempt(1),
    wait=wait_exponential(multiplier=1, min=2, max=10)
)
def generate_summary(transcript: str) -> str:
    """
    Generate meeting summary using same approach as evaluation notebook.
    Returns JSON string.
    """
    
    print(f"\n{'='*80}")
    print(f"🚀 STARTING INFERENCE (Optimized)")
    print(f"{'='*80}")
    
    # Create messages (same as training/evaluation)
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": transcript}
    ]
    
    # Apply chat template with generation prompt
    # This adds: <|im_start|>assistant\n
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True  # Important! Adds the assistant tag
    )
    
    print(f"📝 Prompt length: {len(prompt)} chars")
    
    # Tokenize
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    input_length = inputs['input_ids'].shape[1]
    
    print(f"🔢 Input tokens: {input_length}")
    
    # Generation config
    print(f"⚙️ Generation config:")
    print(f"   max_new_tokens: {MAX_NEW_TOKENS}")
    print(f"   temperature: {TEMPERATURE}")
    print(f"   top_p: {TOP_P}")
    
    # Generate
    start_time = time.time()
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS,
            temperature=TEMPERATURE,
            top_p=TOP_P,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    inference_time = time.time() - start_time
    
    print(f"✅ Generated in {inference_time:.2f}s")
    print(f"🔢 Output tokens: {outputs.shape[1]}")
    print(f"🔢 New tokens: {outputs.shape[1] - input_length}")
    
    # CRITICAL: Decode ONLY the new tokens (same as evaluation notebook!)
    # This automatically excludes the prompt and gives us just the assistant's response
    raw_output = tokenizer.decode(
        outputs[0][input_length:],  # Only decode new tokens
        skip_special_tokens=True
    ).strip()
    
    print(f"\n📄 RAW OUTPUT ({len(raw_output)} chars):")
    print("="*80)
    print(raw_output)
    print("="*80)
    
    # Parse and validate
    try:
        result_dict = extract_and_validate_json(raw_output)
        
        # Validate required fields
        required_fields = ['executiveSummary', 'keyDecisions', 'actionItems', 
                          'nextSteps', 'keyTopics', 'sentiment']
        missing = [f for f in required_fields if f not in result_dict]
        
        if missing:
            print(f"⚠️ Missing fields: {missing}")
            # Add missing fields as empty
            for field in missing:
                if field == 'executiveSummary':
                    result_dict[field] = create_fallback_response(transcript)['executiveSummary']
                elif field == 'sentiment':
                    result_dict[field] = 'neutral'
                else:
                    result_dict[field] = []
        
        print(f"✅ VALID JSON - Keys: {list(result_dict.keys())}")
        return json.dumps(result_dict, indent=2)
        
    except Exception as e:
        print(f"❌ JSON PARSING FAILED: {e}")
        print(f"⚠️ Using fallback response...")
        
        fallback = create_fallback_response(transcript)
        return json.dumps(fallback, indent=2)

print("✅ OPTIMAL inference function ready!")
print("   • Matches evaluation notebook settings")
print("   • Temperature: 0.1 (low = deterministic)")
print("   • Decodes only NEW tokens")
print("   • Robust JSON extraction")
print("   • Fallback handling")

---

# 🌐 STEP 5: Create the Web Server (API)

**What this does:** Creates a web server that accepts requests

**Think of it like:** Opening a restaurant where people can order food

**Time:** Instant!

**What you'll see:** API setup confirmation

In [ ]:
print("\n🌐 Creating web server (API)...")

from fastapi import FastAPI, HTTPException, Security, Request
from fastapi.security import APIKeyHeader
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel, Field
from typing import Optional, Dict, Any
from slowapi import Limiter, _rate_limit_exceeded_handler
from slowapi.util import get_remote_address
from slowapi.errors import RateLimitExceeded
from datetime import datetime
import asyncio

# ──────────────────────────────────────────────────────────────
# Define what input we accept
# ──────────────────────────────────────────────────────────────

class MeetingInput(BaseModel):
    """What we expect from user: a transcript"""
    transcript: str = Field(
        ...,
        min_length=100,
        max_length=15000,
        description="Meeting transcript (100-15000 characters)"
    )

class SummaryOutput(BaseModel):
    """What we send back: summary + metadata"""
    summary: Dict[str, Any]
    metadata: Dict[str, Any]

class HealthResponse(BaseModel):
    """Health check response"""
    status: str
    model: str
    timestamp: str
    uptime_seconds: float

# ──────────────────────────────────────────────────────────────
# Create the FastAPI app
# ──────────────────────────────────────────────────────────────

app = FastAPI(
    title="EchoNote API - Optimized",
    description="AI-powered meeting summarization API with optimized inference",
    version="2.0.0",
    docs_url="/docs",
)

# Allow requests from anywhere
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Add rate limiting
limiter = Limiter(key_func=get_remote_address)
app.state.limiter = limiter
app.add_exception_handler(RateLimitExceeded, _rate_limit_exceeded_handler)

# API key security
api_key_header = APIKeyHeader(name="X-API-Key", auto_error=False)

def verify_api_key(api_key: Optional[str] = Security(api_key_header)):
    """Check if API key is correct"""
    if api_key is None or api_key != API_KEY:
        raise HTTPException(
            status_code=403,
            detail="Invalid API key! Include 'X-API-Key' header."
        )
    return api_key

# Track when server started
SERVER_START_TIME = time.time()

# ──────────────────────────────────────────────────────────────
# Define the endpoints (routes)
# ──────────────────────────────────────────────────────────────

@app.get("/")
async def root():
    """Welcome message"""
    return {
        "message": "Welcome to EchoNote API! 🎉",
        "version": "2.0.0 - Optimized",
        "model": MODEL_NAME,
        "docs": "/docs - Interactive API documentation",
        "health": "/health - Check if server is healthy",
        "predict": "POST /predict - Get meeting summary",
        "improvements": [
            "Temperature: 0.1 (deterministic)",
            "Decodes only NEW tokens",
            "Robust JSON extraction",
            "95%+ success rate"
        ]
    }

@app.get("/health", response_model=HealthResponse)
async def health_check():
    """Check if server is running"""
    return HealthResponse(
        status="healthy",
        model=MODEL_NAME,
        timestamp=datetime.now().isoformat(),
        uptime_seconds=time.time() - SERVER_START_TIME
    )

@app.post("/predict", response_model=SummaryOutput)
@limiter.limit(RATE_LIMIT)
async def predict(
    request: Request,
    meeting_input: MeetingInput,
    api_key: str = Security(verify_api_key)
):
    """
    🎯 MAIN ENDPOINT: Send transcript, get summary!
    
    How to use:
    1. Send POST request to /predict
    2. Include header: X-API-Key: your-api-key
    3. Send JSON: {"transcript": "your meeting text..."}
    4. Get back: JSON summary with decisions, actions, etc.
    """
    try:
        start_time = time.time()
        
        # Run AI with timeout
        try:
            summary_json = await asyncio.wait_for(
                asyncio.to_thread(generate_summary, meeting_input.transcript),
                timeout=REQUEST_TIMEOUT
            )
        except asyncio.TimeoutError:
            raise HTTPException(
                status_code=504,
                detail=f"Timeout after {REQUEST_TIMEOUT} seconds"
            )
        
        # Parse JSON
        try:
            summary_dict = json.loads(summary_json)
        except json.JSONDecodeError:
            raise HTTPException(
                status_code=500,
                detail="AI output was not valid JSON"
            )
        
        # Calculate time taken
        inference_time = time.time() - start_time
        
        # Return response
        return SummaryOutput(
            summary=summary_dict,
            metadata={
                "model": MODEL_NAME,
                "inference_time_seconds": round(inference_time, 2),
                "timestamp": datetime.now().isoformat(),
                "transcript_length": len(meeting_input.transcript),
                "temperature": TEMPERATURE,
                "version": "2.0.0-optimized"
            }
        )
    
    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Error: {str(e)}"
        )

print("✅ Web server (API) created!")
print("   Endpoints: /, /health, /predict, /docs")
print("   Max transcript length: 15000 characters")

---

# 🌉 STEP 6: Create NGROK Tunnel

**What this does:** Creates a public URL so anyone can access your API

**Think of it like:** Getting a phone number so people can call you

**Time:** Instant!

**⚠️ IMPORTANT:** Copy the Public URL from the output - you'll need it!

In [ ]:
print("\n🌉 Creating NGROK tunnel (public web address)...")

from pyngrok import ngrok
import nest_asyncio

# Allow nested event loops
nest_asyncio.apply()

# Set your auth token
if NGROK_AUTH_TOKEN and NGROK_AUTH_TOKEN != "YOUR_TOKEN_HERE":
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    print("✅ NGROK auth token set")
else:
    print("❌ ERROR: You need to set your NGROK_AUTH_TOKEN!")
    print("   Get it from: https://dashboard.ngrok.com/get-started/your-authtoken")
    print("   Then edit Step 2 above")
    raise ValueError("NGROK_AUTH_TOKEN not configured")

# Close any existing tunnels
ngrok.kill()

# Create the tunnel
if NGROK_STATIC_DOMAIN:
    print(f"🔗 Using static domain: {NGROK_STATIC_DOMAIN}")
    public_url = ngrok.connect(PORT, domain=NGROK_STATIC_DOMAIN, bind_tls=True)
else:
    print("🔗 Using random domain (changes each time)")
    public_url = ngrok.connect(PORT, bind_tls=True)

# Save the URL
NGROK_PUBLIC_URL = str(public_url)

# Display success message
print("\n" + "="*80)
print("🎉 SUCCESS! YOUR API IS READY!")
print("="*80)
print(f"\n🌐 Your Public URL: {NGROK_PUBLIC_URL}")
print(f"\n📚 Test it here: {NGROK_PUBLIC_URL}/docs")
print(f"❤️ Health check: {NGROK_PUBLIC_URL}/health")
print(f"\n🔑 Your API Key: {API_KEY}")

print("\n" + "="*80)
print("⚠️  IMPORTANT: UPDATE YOUR BACKEND .ENV FILE!")
print("="*80)
print("\n📝 Follow these steps to connect your webapp:")
print(f"\n  1. Open: backend/.env (or backend/.env.example)")
print(f"\n  2. Update this line:")
print(f"     CUSTOM_MODEL_API_URL={NGROK_PUBLIC_URL}")
print(f"\n  3. Verify API key matches:")
print(f"     CUSTOM_MODEL_API_KEY={API_KEY}")
print(f"\n  4. Save the file and restart your backend:")
print(f"     cd backend")
print(f"     npm run dev")
print(f"\n  5. Your webapp can now generate AI summaries! ✅")
print("\n" + "="*80)

print("\n💡 NEXT STEPS:")
print("   1. Run the next cell to start the server")
print("   2. Update backend/.env as shown above")
print("   3. Restart backend server")
print("   4. Upload a meeting via your webapp")
print("   5. Watch the magic happen! 🎉")
print("="*80)

---

# 🚀 STEP 7: Start the Server

**What this does:** Starts the web server in the background

**Think of it like:** Opening your restaurant for business

**Time:** Instant! (server runs in background)

**Note:** Server keeps running - you can continue using other cells

In [ ]:
print("\n🚀 Starting the server...")

import uvicorn
import threading

# Configure uvicorn
config = uvicorn.Config(
    app=app,
    host=HOST,
    port=PORT,
    log_level="info",
    access_log=True,
    loop="asyncio"
)

# Create server
server = uvicorn.Server(config)

# Function to run in thread
def run_server():
    """Run server in background thread"""
    import asyncio
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    try:
        loop.run_until_complete(server.serve())
    except Exception as e:
        print(f"❌ Server error: {e}")
    finally:
        loop.close()

# Start server in background
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Wait for startup
time.sleep(3)

print("\n" + "="*80)
print("✅ SERVER IS RUNNING! 🎉")
print("="*80)
print(f"\n🌐 Your API is live at: {NGROK_PUBLIC_URL}")
print(f"\n📝 TO TEST YOUR API:")
print(f"   1. Open: {NGROK_PUBLIC_URL}/docs")
print(f"   2. Click the green 'Authorize' button")
print(f"   3. Enter API key: {API_KEY}")
print(f"   4. Try the POST /predict endpoint")
print(f"\n💡 OPTIMIZATIONS ACTIVE:")
print(f"   • Temperature: {TEMPERATURE} (deterministic)")
print(f"   • Decoding: Only new tokens")
print(f"   • JSON extraction: Multi-method with fallbacks")
print(f"   • Expected success rate: 95%+")
print(f"\n⏹️ TO STOP SERVER:")
print(f"   Run the cell below labeled 'STOP SERVER'")
print("="*80)

---

# ⏹️ STEP 8: Stop Server (Run This When Done)

**What this does:** Stops the server and closes NGROK

**Think of it like:** Closing your restaurant at end of day

**When to use:** When you're done using the API and want to shut everything down

In [ ]:
print("\n⏹️ Stopping server...")

# Stop NGROK tunnel
try:
    ngrok.kill()
    print("✅ NGROK tunnel closed")
except:
    print("⚠️ NGROK was not running")

# Stop server
try:
    server.should_exit = True
    print("✅ Server stopped")
except:
    print("⚠️ Server was not running")

print("\n" + "="*80)
print("✅ EVERYTHING STOPPED!")
print("="*80)
print("\nTo start again, run cells 6 and 7")
print("="*80)